In [1]:
import sys
sys.executable

'/home/mgotsmy/anaconda3/bin/python'

# Snek Demo

Snek is developed to improve upon the original cobra package by making it easier to access and less prone for unintended errors.

In [1]:
import cobra 
import snek
model = cobra.io.load_model('textbook')

def printer(x):
    print('{:5.2f}'.format(x))

ModuleNotFoundError: No module named 'snek'

## 1. Ensure Correct Spelling

When setting bounds of a reaction or the objective function with the main CobraPy package, no error is raised when a spelling error occurs.

In [2]:
with model:
    model.reactions.EX_glc__D_e.lower_bound = -5
    printer(model.slim_optimize())
    model.reactions.EX_glc__D_e.lower_buond = -10
    # no error is raised but bound is not updated.
    printer(model.slim_optimize())

 0.42
 0.42


This can happen frequently and is hard to debug. Also, the officially recommended way of setting exchange bounds needs 3 lines instead of 1.

In [3]:
with model:
    medium = model.medium
    medium["EX_glc__D_e"] = 5
    model.medium = medium
    printer(model.slim_optimize())
    medium["EX_glc__D_e"] = 10
    model.medium = medium
    printer(model.slim_optimize())

 0.42
 0.87


The following snek.core functions ensure correct spelling and are still one-liners for efficient coding.

In [4]:
with model:
    snek.set_bounds(model,'EX_glc__D_e',-5)
    printer(model.slim_optimize())
    snek.set_bounds(model,'EX_glc__D_e',-10)
    printer(model.slim_optimize())

 0.42
 0.87


The same issue can arise when trying to change the objective function of a model.

In [5]:
with model:
    printer(model.slim_optimize())
    model.objctive = 'EX_etoh_e'
    # no error is raised but objective function is not updated.
    printer(model.slim_optimize())
    snek.set_objective(model,'EX_etoh_e','max')
    printer(model.slim_optimize())

 0.87
 0.87
20.00


## 2. Check for Unintended Results.

CobraPy gives the user a lot of freedom. However, here we implemented some basic checks that ensure that no unintended are made and prints warning if this is true.
This logical errors are sometimes intended, therefore only warnings are printed but the objective function is still updated. 
However, we think that a more verbose output can help researchers when exploring the model.

In [6]:
with model:
    model.objective = 'ATPM'
    model.objective_direction = 'min'
    printer(model.slim_optimize())
    # here we try to minimize a reaction with a custom lower bound
    # although this might be intended, it often isn't.
    snek.set_objective(model,'ATPM','min')
    printer(model.slim_optimize())

 8.39
 8.39


When the solution status is infeasible it is still possible to programmatically extract fluxes. This can lead to confusions while coding.

In [7]:
with model:
    snek.set_bounds(model,'EX_glc__D_e',0,0)
    solution = model.optimize()
    print(solution.status)
    printer(solution.fluxes['EX_glc__D_e'])

infeasible
-0.48


/home/users/mgotsmy/.conda/envs/2206test3.10/lib/python3.10/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


The snek optimization function checks for infeasibility and raises an error, so no confusions can happen.

In [8]:
with model:
    snek.set_bounds(model,'EX_glc__D_e',0,0)
    solution = snek.sensitive_optimize(model)
    print(solution.status)
    printer(solution.fluxes['EX_glc__D_e'])

/home/users/mgotsmy/.conda/envs/2206test3.10/lib/python3.10/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
ERROR:root:Solution status is infeasible.


TypeError: exceptions must derive from BaseException

## 3. Easy Programmatic Access

Some functionalities can be set very easily by strings, however, when you want the value that was entered returned you get a programmatically hard to deal with object back.

In [9]:
with model:
    model.solver = 'cplex'
    print(str(model.solver)[:100],'\n\n...\n',str(model.solver)[-100:])

\ENCODING=ISO-8859-1
\Problem name: 

Maximize
 b2ff9840m076fm11edm935cmc581d07a87a9#0: Biomass_Ecol 

...
 
 0 <= TKT2_reverse_7ebc7#187 <= 1000
 0 <= TPI#188 <= 1000
 0 <= TPI_reverse_c2c3b#189 <= 1000
End



compared to 

In [10]:
with model:
    model.solver = 'cplex'
    print(snek.get_solver(model))

cplex


The same is implemented for (simple) objectives:

In [11]:
with model:
    model.objective = 'Biomass_Ecoli_core'
    print(str(model.objective.expression))

1.0*Biomass_Ecoli_core - 1.0*Biomass_Ecoli_core_reverse_2cdba


compared to 

In [12]:
with model:
    model.objective = 'Biomass_Ecoli_core'
    print(snek.get_objective(model))

Biomass_Ecoli_core


The summary command calculates the C fluxes, however they cannot be easily programmatically accessed.

In [13]:
with model:
    summary = model.summary()
summary

Metabolite,Reaction,Flux,C-Number,C-Flux
glc__D_e,EX_glc__D_e,10,6,100.00%
nh4_e,EX_nh4_e,4.765,0,0.00%
o2_e,EX_o2_e,21.8,0,0.00%
pi_e,EX_pi_e,3.215,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,-22.81,1,100.00%
h2o_e,EX_h2o_e,-29.18,0,0.00%
h_e,EX_h_e,-17.53,0,0.00%


In [14]:
summary.uptake_flux
# the C-flux is gone

,flux,reaction,metabolite
EX_glc__D_e,10.000000,EX_glc__D_e,glc__D_e
EX_nh4_e,4.765319,EX_nh4_e,nh4_e
EX_o2_e,21.799493,EX_o2_e,o2_e
EX_pi_e,3.214895,EX_pi_e,pi_e


With the ``snek.elements`` module the element wise fluxes can easily be accessed. Moreover, all non mass-balanced fluxes + their absolut values are included.

In [15]:
snek.elements.element_fluxes(model,'C')

,C_flux,C_flux%
EX_glc__D_e,60.000000,100.000000
EX_co2_e,-22.809833,-38.016389
Biomass_Ecoli_core,-37.190167,-61.983611


Moreover, not only C fluxes can be accessed. 

In [16]:
snek.elements.element_fluxes(model,'H')

,H_flux,H_flux%
EX_glc__D_e,120.000000,84.343006
EX_nh4_e,19.061277,13.397378
EX_pi_e,3.214895,2.259616
EX_h_e,-17.530865,-12.321716
EX_h2o_e,-58.351654,-41.012949
Biomass_Ecoli_core,-66.393652,-46.665335


``snek.elements`` also give easier access to molecular weight calculations:

In [17]:
some_metabolite = cobra.Metabolite()
some_metabolite.formula = 'C6H12O6'
printer(some_metabolite.formula_weight)

printer(snek.elements.molecular_weight('C6H12O6'))

180.16
180.16


Compared to cobraPy's model.summary(), snek returns sorted data frames with the sign of reactions fluxes not the metabolite fluxes. (I guess this is a preference issue.)

In [18]:
summary = model.summary()
summary.uptake_flux

,flux,reaction,metabolite
EX_glc__D_e,10.000000,EX_glc__D_e,glc__D_e
EX_nh4_e,4.765319,EX_nh4_e,nh4_e
EX_o2_e,21.799493,EX_o2_e,o2_e
EX_pi_e,3.214895,EX_pi_e,pi_e


In [19]:
snek.analysis.in_flux(model)

,name,id,flux
EX_o2_e,O2 exchange,EX_o2_e,-21.799493
EX_glc__D_e,D-Glucose exchange,EX_glc__D_e,-10.0
EX_nh4_e,Ammonia exchange,EX_nh4_e,-4.765319
EX_pi_e,Phosphate exchange,EX_pi_e,-3.214895


for whatever reason the cobraPy summary.secretion_flux gives also 0 flux values (in contrast to the in_flux):

In [20]:
summary.secretion_flux

,flux,reaction,metabolite
EX_ac_e,0.000000,EX_ac_e,ac_e
EX_acald_e,0.000000,EX_acald_e,acald_e
EX_akg_e,0.000000,EX_akg_e,akg_e
EX_co2_e,-22.809833,EX_co2_e,co2_e
EX_etoh_e,0.000000,EX_etoh_e,etoh_e
EX_for_e,0.000000,EX_for_e,for_e
EX_fru_e,0.000000,EX_fru_e,fru_e
EX_fum_e,0.000000,EX_fum_e,fum_e
EX_gln__L_e,0.000000,EX_gln__L_e,gln__L_e
EX_glu__L_e,0.000000,EX_glu__L_e,glu__L_e


``snek.analysis.out_flux`` only returns non-0 fluxes with the sign of reaction fluxes.

In [21]:
snek.analysis.out_flux(model)

,name,id,flux
EX_h2o_e,H2O exchange,EX_h2o_e,29.175827
EX_co2_e,CO2 exchange,EX_co2_e,22.809833
EX_h_e,H+ exchange,EX_h_e,17.530865


## 4. New Functions

There are some additional functionalities that are almost always handy. E.g a function that looks for biomass reactions:

In [22]:
snek.find_biomass_reaction(model)

Biomass_Ecoli_core


Altough there are search functions in CobraPy, here we implemented some easy to use functions to find reactions and metabolites

In [23]:
snek.find_reaction(model,'glucose')

,id,name
0,EX_glc__D_e,D-Glucose exchange
1,G6PDH2r,glucose 6-phosphate dehydrogenase
2,GLCpts,D-glucose transport via PEP:Pyr PTS
3,PGI,glucose-6-phosphate isomerase


In [24]:
snek.find_metabolite(model,'coa')

,id,name
0,accoa_c,Acetyl-CoA
1,coa_c,Coenzyme A
2,succoa_c,Succinyl-CoA


Moreover, when opening a new CobraPy model, one usually wants to check if there are unusual (i.e., constrained) bounds. There is no good default functionality.

In [25]:
snek.analysis.get_constrained_reactions(model)

,name,id,lower_bound,upper_bound
ATPM,ATP maintenance requirement,ATPM,8.39,1000.0
EX_glc__D_e,D-Glucose exchange,EX_glc__D_e,-10.00,1000.0


I think it is nice to be able to look up where fluxes are going, so I implemented a function that shows the local topology of the metabolic network. This basically prints out where metabolites end up at. Numbers are levels of distance to the initial reaction node. Letters are different branches. > denotes metabolites in the reactions. Numbers in parenthesis represent fluxes (reactions) or stoichiometry (metabolites).

In [26]:
solution = snek.sensitive_optimize(model,pFBA=True)

snek.analysis.investigate_network_solution(model,solution,'EX_glc__D_e',2)

1.A   EX_glc__D_e       (-10.00000)
      > glc__D_e        ( -1.0)
2.A   GLCpts            (10.00000)
      > glc__D_e        ( -1.0)
      > pep_c           ( -1.0)
      > g6p_c           (  1.0)
      > pyr_c           (  1.0)


E.g. here, glucose is imported by GLCpts that directly phosphorylates glucose.
This function still needs some polishing. It is possible to print nodes with higher distance as well:

In [27]:
snek.analysis.investigate_network_solution(model,solution,'EX_pi_e',4)

1.A   EX_pi_e           (-3.21490)
      > pi_e            ( -1.0)
2.A   PIt2r             (3.21490)
      > h_e             ( -1.0)
      > pi_e            ( -1.0)
3.C   CYTBD             (43.59899)
      > o2_c            ( -0.5)
      > q8h2_c          ( -1.0)
      > h_e             (  2.0)
      > q8_c            (  1.0)
3.G   EX_h_e            (17.53087)
      > h_e             ( -1.0)
3.I   ATPS4r            (45.51401)
      > h_e             ( -4.0)
3.K   NADH16            (38.53461)
      > nadh_c          ( -1.0)
      > q8_c            ( -1.0)
      > h_e             (  3.0)
      > nad_c           (  1.0)
      > q8h2_c          (  1.0)
4.D   AKGDH             (5.06438)
      > akg_c           ( -1.0)
      > coa_c           ( -1.0)
      > nad_c           ( -1.0)
      > co2_c           (  1.0)
      > nadh_c          (  1.0)
      > succoa_c        (  1.0)
4.E   PDH               (9.28253)
      > coa_c           ( -1.0)
      > nad_c           ( -1.0)
      > pyr_c      